In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import tensorflow as tf 

In [ ]:

print('tensorflow version', tf.__version__)
x = [[3.]]
y = [[4.]]
print('Result: {}'.format(tf.matmul(x, y)))

tensorflow version 2.12.0
Result: [[12.]]


In [ ]:
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/data.zip"  -d "/content/data/"

In [ ]:
# data :- train.csv
import pandas as pd
# loading csv data file :-- for test and train
train=pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test.csv')


In [ ]:
# Small details:--
train.head()

,Image,Label
0,YGP5FW.jpg,0
1,R5BWPN.jpg,0
2,ESROUO.jpg,0
3,4CYU0L.jpg,0
4,3WG532.jpg,0


In [ ]:
#we can create batches of images for the above images, and setting target as label
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [ ]:
train['Label']=train['Label'].astype('str')

In [ ]:
train_set = train_datagen.flow_from_dataframe(train,
                                              directory="/content/data/train/",
                                              x_col="Image",
                                              y_col="Label",
                                              target_size=(150, 150),
                                              batch_size=32,
                                             class_mode='binary'
                                              )

Found 15933 validated image filenames belonging to 2 classes.


In [ ]:
test['Label']=test['Label'].astype('str')

In [ ]:
test_set = train_datagen.flow_from_dataframe(test,
                                              directory="/content/data/test/",
                                              x_col="Image",
                                              y_col="Label",
                                              target_size=(150, 150),
                                              batch_size=32
                                              )

Found 6829 validated image filenames belonging to 1 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # since Conv2D is the first layer of the neural network, we should also specify the size of the input
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    # apply pooling
    tf.keras.layers.MaxPooling2D(2,2),
    # and repeat the process
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten the result to feed it to the dense layer
    tf.keras.layers.Flatten(), 
    # and define 512 neurons for processing the output coming by the previous layers
    tf.keras.layers.Dense(512, activation='relu'), 
    # a single output neuron. The result will be 0 if the image is a cat, 1 if it is a dog
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
import numpy as np
epochs=10
with tf.device('/GPU:0'):
    history = model.fit_generator(
        train_set,
        steps_per_epoch=15933//32,
        epochs=epochs
    )

Epoch 1/10
497/497 [==============================] - 158s 312ms/step - loss: 0.6873 - accuracy: 0.5675
Epoch 2/10
497/497 [==============================] - 153s 307ms/step - loss: 0.6769 - accuracy: 0.5777
Epoch 3/10
497/497 [==============================] - 153s 308ms/step - loss: 0.6792 - accuracy: 0.5769
Epoch 4/10
497/497 [==============================] - 152s 306ms/step - loss: 0.6713 - accuracy: 0.5880
Epoch 5/10
497/497 [==============================] - 151s 304ms/step - loss: 0.6673 - accuracy: 0.5956
Epoch 6/10
497/497 [==============================] - 150s 301ms/step - loss: 0.6625 - accuracy: 0.5995
Epoch 7/10
497/497 [==============================] - 151s 303ms/step - loss: 0.6589 - accuracy: 0.6085
Epoch 8/10
497/497 [==============================] - 152s 306ms/step - loss: 0.6569 - accuracy: 0.6091
Epoch 9/10
497/497 [==============================] - 150s 302ms/step - loss: 0.6529 - accuracy: 0.6149
Epoch 10/10
497/497 [==============================] - 150s 302m

In [ ]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/water.h5')
print('Model Saved!')
 
# load model
savedModel=load_model('/content/drive/MyDrive/water.h5')
savedModel.summary()

Model Saved!
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 17, 

In [ ]:
model.save_weights('/content/drive/MyDrive/water_weights.h5')
print('Model Saved!')

Model Saved!


In [ ]:
preds=model.predict(test_set)

214/214 [==============================] - 63s 293ms/step


In [ ]:
value=[]
for i in preds:
  if i > 0.5:
    value.append(1)
  else:
    value.append(0)

In [ ]:
len(value)

6829

In [ ]:
df=pd.DataFrame(value, columns=['Labels'])

In [ ]:
df.to_csv('submission.csv')

In [ ]:
df.to_csv('my_submission_file.csv', index=False)